In [0]:
   ! ls drive/Проверено/Проверено_Плотниковой/Набокова_кодТимофеева_проверено\ Плотниковой

25106701_435248.xlsx  25106701_436015.xlsx    25106701_436843.xlsx
25106701_435295.xlsx  25106701_436141.xlsx    25106701_437177.xlsx
25106701_435302.xlsx  25106701_436161.xlsx    25106701_437195.xlsx
25106701_435600.xlsx  ~$25106701_436183.xlsx  25106701_437261.xlsx
25106701_435736.xlsx  25106701_436183.xlsx    25106701_437284.xlsx
25106701_435748.xlsx  25106701_436335.xlsx    25106701_437375.xlsx
25106701_435879.xlsx  25106701_436405.xlsx    25106701_437378.xlsx
25106701_435913.xlsx  25106701_436539.xlsx    25106701_437410.xlsx
25106701_435965.xlsx  25106701_436624.xlsx    25106701_437551.xlsx


In [60]:
! pip install python-igraph
! pip install xlrd

In [0]:
import pandas as pd
import math
import numpy as np
import igraph
import os

def xlsx_to_gml(filename):
    #read xlsx file into dataframe
    post_df = pd.read_excel(filename)
    #list to add comment ids that were deleted for some reason
    print(filename)
    #creating a root node
    root = {'id' : filename[len(filename)-11:len(filename)-5]}
    root_df = pd.DataFrame(root,index = ['0'])
    
    
    post_df = pd.concat([root_df,post_df],ignore_index=True)
    
    if 'attachments' not in post_df.columns:
      post_df['attachments'] = ''
    if 'reply_to_comment' not in post_df.columns:
      post_df['reply_to_comment'] = 0
    if 'reply_to_user' not in post_df.columns:
      post_df['reply_to_user'] = 0
    
    met_id = list()

    for comment in post_df.iterrows():
        if ( (not math.isnan(comment[1]['reply_to_comment'])) and 
            (comment[1]['reply_to_comment'] not in met_id) and
            (comment[1]['reply_to_comment'] not in post_df['id'].tolist()) ):
                met_id.append(comment[1]['reply_to_comment'])
                
    
    met_id_df = pd.DataFrame({'id' : met_id})
    #adding empty rows with id field so that every comment id mentioned in xlsx gets its node.
    post_df = pd.concat([post_df,met_id_df],ignore_index=True)

    

    post_Graph = igraph.Graph(directed=True)
    post_Graph.add_vertices(len(post_df)) #all comments get a node + root node for the post itself

    post_Graph.vs['attachments'] = post_df['attachments'].fillna('').apply(str).tolist()
    post_Graph.vs['date'] = post_df['date'].fillna(0).apply(int).apply(lambda x: x if x > 0 else '').apply(str).tolist()
    post_Graph.vs['from_id'] = post_df['from_id'].fillna(0).apply(int).apply(lambda x: x if x > 0 else '').apply(str).tolist()
    post_Graph.vs['comment_id'] = post_df['id'].fillna(0).apply(int).apply(lambda x: x if x > 0 else '').apply(str).tolist()
    post_Graph.vs['reply_to_comment'] = post_df['reply_to_comment'].fillna(0).apply(int).apply(lambda x: x if x > 0 else '').apply(str).tolist()
    post_Graph.vs['reply_to_user'] = post_df['reply_to_user'].fillna(0).apply(int).apply(lambda x: x if x > 0 else '').apply(str).tolist()
    post_Graph.vs['text'] = post_df['text'].fillna('').apply(str).tolist()
    post_Graph.vs['intent'] = post_df['Intent'].fillna('').apply(str).tolist()
    post_Graph.vs['content'] = post_df['Content'].replace(' -',float('NaN')).fillna('').apply(str).tolist()
    post_Graph.vs['intervention'] = post_df['Intervention'].fillna('').apply(str).tolist()
    
    for comment in post_df.iterrows():
        if ( (math.isnan(comment[1]['reply_to_comment']))):
            post_Graph.add_edge(0,comment[0])
        else:
            for replied_comment in post_df.iterrows():
                if (replied_comment[1]['id'] == comment[1]['reply_to_comment']):
                    post_Graph.add_edge(replied_comment[0], comment[0])
    
    
    post_Graph.write_gml( (filename[:6] + 'gml_from_excel' + filename[15:len(filename)-5] + '.gml').replace(' ',''))
    #post_Graph.write_gml('drive/gml_from_excel/' + '25106701_437551' +'.gml')
    print('gml is done')
    return post_Graph

In [0]:
import os
filenames = list()
for directory, subdirectories, files in os.walk('drive/Проверено'):
    for file in files:
        filenames.append(os.path.join(directory,file))
for filename in filenames:
  filename = filename.replace(" ", " ")
  xlsx_to_gml(filename)


In [0]:
import pandas as pd
import numpy as np
kek_list = list()
kek_list.append('45')
kek_list.append('42352342')

kek_df = pd.DataFrame({'id' : kek_list})
data = np.array([np.arange(10)+1]*3).T
topkek_df = pd.DataFrame(data, columns = ['id','name','intent'])
topkek_df = topkek_df.append(kek_df, ignore_index = True)
len(topkek_df)

12

In [2]:
# Install requirements 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Connect
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [85]:
print(post_Graph)
#print(post_Graph.subgraph( (3,4,7,8) ))
for v in post_Graph.vs.select(intervention_ne=''):
  print(v.index)
  subgraph_vertices = list()
  subgraph_vertices.append(v.index)
  neighbors = (v.successors())
  print(neighbors)
  print('Constructing subgraph...')
  while len(neighbors) > 0:
    for neighbor in neighbors:
      print(neighbor)
      neighbors.extend(neighbor.successors())
      subgraph_vertices.append(neighbor.index)
      neighbors.remove(neighbor)
  print(subgraph_vertices)

IGRAPH D--- 42 42 --
+ attr: attachments (v), comment_id (v), content (v), date (v), from_id (v),
  intent (v), intervention (v), reply_to_comment (v), reply_to_user (v), text
  (v)
+ edges:
0->0 0->1 0->2 1->3 3->4 3->5 0->6 4->7 7->8 5->9 9->10 10->11 0->12 12->13
13->14 14->15 15->16 3->17 16->18 0->19 0->20 19->21 0->22 0->23 0->24 12->25
14->26 0->27 0->28 0->29 0->30 30->31 31->32 32->33 33->34 33->35 0->36 36->37
37->38 38->39 39->40 40->41
29
[]
Constructing subgraph...
[29]
31
[igraph.Vertex(<igraph.Graph object at 0x7f7ea0d80318>, 32, {'attachments': '', 'date': '1503273002', 'from_id': '212488009', 'comment_id': '437790', 'reply_to_comment': '437705', 'reply_to_user': '437072715', 'text': '[id437072715|Finest], В соответствии с приказом императора Николая II о создании Кавказской туземной конной дивизии от 23 августа 1914 года дивизию составляли управление и три бригады по два кавказских туземных конных полка (каждый в 4 эскадрона). В состав дивизии входили следующие формиро

In [62]:
post_Graph = xlsx_to_gml('drive/Проверено/Проверено_Плотниковой/Набокова_кодКудрявцевой_провереноПлотниковой/25106701_437584.xlsx')



drive/Проверено/Проверено_Плотниковой/Набокова_кодКудрявцевой_провереноПлотниковой/25106701_437584.xlsx
gml is done


In [0]:
filename = ('drive/Проверено/Проверено_Плотниковой/Набокова_кодКудрявцевой_провереноПлотниковой/25106701_437584.xlsx')
filename = filename[len(filename)-11:len(filename)-5]
filename

'437584'

In [63]:
print(post_Graph.subgraph((29,30,31,32)))

IGRAPH D--- 4 2 --
+ attr: attachments (v), comment_id (v), content (v), date (v), from_id (v),
  intent (v), intervention (v), reply_to_comment (v), reply_to_user (v), text
  (v)
+ edges:
1->2 2->3


In [0]:
for directory, subdirectories, files in os.walk('drive/Проверено'):
    for file in files:
        filenames.append(os.path.join(directory,file))
print(filenames)

['drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_441061.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_441003.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440568.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440697.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_441192.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440603.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_441157.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440544.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440664.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_441680.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Плотникова_кодНабоковой/25106701_440883.xlsx', 'drive/Проверено/Проверено_Кудрявцевой /Пл

In [47]:
--NotebookApp.iopub_data_rate_limit

NameError: ignored